In [1]:
from db_commands import connect_db

conn = connect_db("olist.db")
cursor = conn.cursor()

 ! Database connected: olist.db


In [19]:
cursor.execute("""
SELECT products.product_id, products.product_length_cm, products.product_height_cm, products.product_width_cm, product_category_translation.product_category_name_english
FROM products
JOIN product_category_translation ON products.product_category_name = product_category_translation.product_category_name;
"""
)
tables = cursor.fetchall()

In [20]:
tables

[('1e9e8ef04dbcff4541ed26657ea517e5', 16.0, 10.0, 14.0, 'perfumery'),
 ('3aa071139cb16b67ca9e5dea641aaa2f', 30.0, 18.0, 20.0, 'art'),
 ('96bd76ec8810374ed1b65e291975717f', 18.0, 9.0, 15.0, 'sports_leisure'),
 ('cef67bcfe19066a932b7673e239eb23d', 26.0, 4.0, 26.0, 'baby'),
 ('9dc1a7de274444849c219cff195d0b71', 20.0, 17.0, 13.0, 'housewares'),
 ('41d3672d4792049fa1779bb35283ed13', 38.0, 5.0, 11.0, 'musical_instruments'),
 ('732bd381ad09e530fe0a5f457d81becb', 70.0, 24.0, 44.0, 'cool_stuff'),
 ('2548af3e6e77a690cf3eb6368e9ab61e', 40.0, 8.0, 40.0, 'furniture_decor'),
 ('37cc742be07708b53a98702e77a21a02', 27.0, 13.0, 17.0, 'home_appliances'),
 ('8c92109888e8cdf9d66dc7e463025574', 17.0, 10.0, 12.0, 'toys'),
 ('14aa47b7fe5c25522b47b4b29c98dcb9', 16.0, 10.0, 16.0, 'bed_bath_table'),
 ('03b63c5fc16691530586ae020c345514', 50.0, 19.0, 45.0, 'baby'),
 ('cf55509ea8edaaac1d28fdb16e48fc22', 17.0, 7.0, 17.0, 'musical_instruments'),
 ('7bb6f29c2be57716194f96496660c7c2', 68.0, 11.0, 13.0, 'furniture_decor

In [68]:
cursor.execute("""
SELECT * FROM orders;
"""
)
tables = cursor.fetchall()

In [69]:
tables

[('e481f51cbdc54678b7cc49136f2d6af7',
  '9ef432eb6251297304e76186b10a928d',
  'delivered',
  '2017-10-02 10:56:33',
  '2017-10-02 11:07:15',
  '2017-10-04 19:55:00',
  '2017-10-10 21:25:13',
  '2017-10-18 00:00:00'),
 ('53cdb2fc8bc7dce0b6741e2150273451',
  'b0830fb4747a6c6d20dea0b8c802d7ef',
  'delivered',
  '2018-07-24 20:41:37',
  '2018-07-26 03:24:27',
  '2018-07-26 14:31:00',
  '2018-08-07 15:27:45',
  '2018-08-13 00:00:00'),
 ('47770eb9100c2d0c44946d9cf07ec65d',
  '41ce2a54c0b03bf3443c3d931a367089',
  'delivered',
  '2018-08-08 08:38:49',
  '2018-08-08 08:55:23',
  '2018-08-08 13:50:00',
  '2018-08-17 18:06:29',
  '2018-09-04 00:00:00'),
 ('949d5b44dbf5de918fe9c16f97b45f8a',
  'f88197465ea7920adcdbec7375364d82',
  'delivered',
  '2017-11-18 19:28:06',
  '2017-11-18 19:45:59',
  '2017-11-22 13:39:59',
  '2017-12-02 00:28:42',
  '2017-12-15 00:00:00'),
 ('ad21c59c0840e6cb83a9ceb5573f8159',
  '8ab97904e6daea8866dbdbc4fb7aad2c',
  'delivered',
  '2018-02-13 21:18:39',
  '2018-02-13 22

In [57]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name")
tables = cursor.fetchall()

for table in tables:
    table_name = table[0]
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    count = cursor.fetchone()[0]
    print(f"  {table_name}: {count:,} rows")

  customers: 99,441 rows
  geolocation: 1,000,163 rows
  order_items: 112,650 rows
  order_payments: 103,886 rows
  order_reviews: 99,224 rows
  orders: 99,441 rows
  product_category_translation: 71 rows
  products: 32,951 rows
  sellers: 3,095 rows
